In [2]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.5 MB/s eta 0:00:00


In [3]:
!pip install neattext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 2.9 MB/s eta 0:00:00


#Import necessary libraries

In [4]:
import pandas as pd
import numpy as np
import datetime as dt
import neattext.functions as nfx
from google.colab import drive
import matplotlib.pyplot as plt
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
import string

#Mount Google Drive


In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


#Define a function to handle bad lines in the CSV file

In [6]:
def bad_line(x):
  print(x)
  return None

#Load the dataset

In [7]:
df_data = pd.read_csv('/content/drive/Othercomputers/My Laptop (1)/year4/final_project/data_science/data_set/final_combined.csv', on_bad_lines=bad_line, engine='python')
print(df_data.shape)

(1529906, 22)


#Define a regular expression pattern that matches English letters

In [8]:
english_pattern = re.compile(r'[a-zA-Z]')
#Use str.contains() method to keep only rows where the 'Tweet' column contains English letters
df_data = df_data[df_data['Tweet'].str.contains(english_pattern)]

#Define a function to clean the text

In [9]:
def clean_text(text):
  # Remove URLs
  text = re.sub(r'http\S+', '', text)
  # Remove mentions (@)
  text = re.sub(r'@\w+', '', text)
  # Remove hashtags (#)
  text = re.sub(r'#\w+', '', text)
  # Remove punctuation
  text = text.translate(str.maketrans('', '', string.punctuation))
  # Convert to lowercase
  text = text.lower()
  # Remove extra spaces
  text = re.sub(r'\s+', ' ', text).strip()
  return text

#Clean the 'Tweet' column and create a new column 'clean_tweet'

In [10]:
df_data['clean_tweet'] = df_data['Tweet'].apply(clean_text)
# Remove white spaces and new line characters from 'clean_tweet'
df_data['clean_tweet'] = df_data['clean_tweet'].apply(nfx.remove_multiple_spaces)

#Define a function to calculate subjectivity of a tweet

In [11]:
def get_subjectivity(tweet):
  return TextBlob(tweet).sentiment.subjectivity

#Define a function to calculate polarity of a tweet

In [12]:
def get_polarity(tweet):
  return TextBlob(tweet).sentiment.polarity
print(df_data.shape)

(1507978, 23)


#Create two columns for subjectivity and polarity

In [13]:
df_data['subjectivity_tb'] = df_data['clean_tweet'].apply(get_subjectivity)
df_data['polarity_tb'] = df_data['clean_tweet'].apply(get_polarity)

#Define a function to get the sentiment of a tweet based on its polarity score

In [14]:
def getSentiment(score):
  if (score < 0 ):
    return 'negative'
  elif (score == 0):
    return 'neutral'
  else:
    return 'positive'

#Create a new column 'sentiment' to store the sentiment of each tweet

In [15]:
df_data['sentiment_tb'] = df_data['polarity_tb'].apply(getSentiment)

In [ ]:
df_data.head(2)

,Tweet,Date,time,Day of week,Cashtags,Hashtags,Language,Location,Mentioned_users,Followers,...,Average_favourite_count,account_age,Likes,Comments,Retweets,Views,clean_tweet,subjectivity_tb,polarity_tb,sentiment_tb
0,@RockNRoLL_85 Apples &amp; oranges. Eddie's g...,2023-03-25 23:59:58,23:59:58,Saturday,NaN,NaN,en,False,True,220.0,...,1.559077,11.0,0.0,0.0,0.0,9.0,apples amp oranges eddies guitar riffs are the...,0.900000,0.60,positive
1,I trust Apple weather app with my life and I j...,2023-03-25 23:59:55,23:59:55,Saturday,NaN,NaN,en,True,False,168.0,...,0.177647,9.0,0.0,0.0,0.0,108.0,i trust apple weather app with my life and i j...,0.691667,0.05,positive


In [ ]:

#function to get the sentiment score from VADER
def get_vader_sentiment_score(tweet):
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(tweet)
    return sentiment

#extract the compound score from the sentiment dictionary returned by VADER
df_data['vader_sentiment'] = df_data['clean_tweet'].apply(get_vader_sentiment_score)
df_data['vader_sentiment'] = df_data['vader_sentiment'].apply(lambda x: x['compound'])

#convert vader sentiment score to sentiment category
def get_vader_sentiment(score):
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

df_data['vader_sentiment_category'] = df_data['vader_sentiment'].apply(get_vader_sentiment)


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate accuracy, precision, recall, and F1-score for TextBlob sentiment analysis
tb_accuracy = accuracy_score(df_data['sentiment'], np.where(df_data['polarity_tb']>0, 'positive', np.where(df_data['polarity_tb']<0, 'negative', 'neutral')))
tb_precision = precision_score(df_data['sentiment'], np.where(df_data['polarity_tb']>0, 'positive', np.where(df_data['polarity_tb']<0, 'negative', 'neutral')), average='weighted')
tb_recall = recall_score(df_data['sentiment'], np.where(df_data['polarity_tb']>0, 'positive', np.where(df_data['polarity_tb']<0, 'negative', 'neutral')), average='weighted')
tb_f1 = f1_score(df_data['sentiment'], np.where(df_data['polarity_tb']>0, 'positive', np.where(df_data['polarity_tb']<0, 'negative', 'neutral')), average='weighted')

print("TextBlob Accuracy:", tb_accuracy)
print("TextBlob Precision:", tb_precision)
print("TextBlob Recall:", tb_recall)
print("TextBlob F1-score:", tb_f1)

# Calculate accuracy, precision, recall, and F1-score for VADER sentiment analysis
vader_accuracy = accuracy_score(df_data['sentiment'], df_data['vader_sentiment_category'])
vader_precision = precision_score(df_data['sentiment'], df_data['vader_sentiment_category'], average='weighted')
vader_recall = recall_score(df_data['sentiment'], df_data['vader_sentiment_category'], average='weighted')
vader_f1 = f1_score(df_data['sentiment'], df_data['vader_sentiment_category'], average='weighted')

print("VADER Accuracy:", vader_accuracy)
print("VADER Precision:", vader_precision)
print("VADER Recall:", vader_recall)
print("VADER F1-score:", vader_f1)

#Save the dataset 

In [ ]:
# save the DataFrame to a CSV file
df_data.to_csv('data_sentiment_1.csv', index=False)